# Configuración e imports

In [1]:
import sys
import os

In [2]:
# Obtener ruta absoluta del directorio que contiene el notebook
notebook_dir = os.path.dirname(os.getcwd())  # sube un nivel desde /notebook
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

In [3]:
from environment.wind_field import MultiDayWindField
from environment.random_wind_wrapper import RandomizedWindWrapper
from environment.random_start_goal_wrapper import RandomStartGoalWrapper
from environment.sailing_env import SailingEnv
from environment.polar_diagram import PolarDiagram
from agents.ddpg import DDPGAgent
from training.train_drl import train_ddpg

2025-07-03 11:37:19.361873: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 11:37:19.379129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751542639.399045 2021923 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751542639.405057 2021923 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751542639.420388 2021923 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
import tensorflow as tf
print("Versión de TensorFlow:", tf.__version__)

Versión de TensorFlow: 2.19.0


In [5]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detectada: {gpus[0].name}")
else:
    print("⚠️ No se ha detectado GPU. El entrenamiento usará CPU.")

✅ GPU detectada: /physical_device:GPU:0


# Entrenamiento modelo

## Carga de datos

In [6]:
# Diagrama polar
polar = PolarDiagram('../../data/polar_diagram.csv')

# Carpeta de rutas CSV del profesor
folder_path = "../../data/expert_trajectories"

In [7]:
# Definir rutas por fecha (ajusta con tus archivos)
csv_paths = {
    "2025-05-04": "../../data/processed/nodes_bathy_wind20250504.csv",
    "2025-05-04": "../../data/processed/nodes_bathy_wind20250507.csv",
    "2025-05-09": "../../data/processed/nodes_bathy_wind20250509.csv",
    "2025-05-11": "../../data/processed/nodes_bathy_wind20250511.csv",
    "2025-05-13": "../../data/processed/nodes_bathy_wind20250513.csv",
    "2025-05-15": "../../data/processed/nodes_bathy_wind20250515.csv",
    "2025-05-17": "../../data/processed/nodes_bathy_wind20250517.csv",
    "2025-05-19": "../../data/processed/nodes_bathy_wind20250519.csv",
    "2025-05-23": "../../data/processed/nodes_bathy_wind20250523.csv",
    "2025-05-25": "../../data/processed/nodes_bathy_wind20250525.csv",
    "2025-05-27": "../../data/processed/nodes_bathy_wind20250527.csv",
    "2025-05-28": "../../data/processed/nodes_bathy_wind20250528.csv",
    "2025-05-30": "../../data/processed/nodes_bathy_wind20250530.csv",
    "2025-06-01": "../../data/processed/nodes_bathy_wind20250601.csv"
}

# Crear campo de viento multi-día
wind_field = MultiDayWindField(csv_paths)

## Inicialización del entorno

- El barco empieza y termina dentro del área navegable.
- El episodio dura como máximo 24 horas.
- Las consultas de viento están cubiertas por tu CSV de Open-Meteo.

In [8]:
# Configuración de entorno simulado
config_base = {
    "dt": 10,
    "max_steps": 135, # rutas de no más de 24 h
    "polar_diagram": polar,
    "debug": True
}

In [9]:
# Crear entorno con viento que cambia por episodio
env = RandomStartGoalWrapper(SailingEnv, config_base, wind_field)

[RUTA] Día: 2025-05-17 | Inicio: [np.float64(40.10318009019184), np.float64(0.657076)] | Destino: [np.float64(39.77205622915002), np.float64(2.5050760000000003)] | Distancia: 161.80 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## Creación del agente

In [10]:
agent = DDPGAgent(env)

I0000 00:00:1751335102.615123 1784309 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46745 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:5b:00.0, compute capability: 8.6


## Entrenamiento

### Entrenamiento con aprendizaje por imitación

In [ ]:
from agents.ddpg import get_actor
from training.imitation_learning import train_actor_supervised

# Crear actor sin entrenar
actor_model = get_actor(input_shape=6, action_bounds=[360, 20])

# Entrenar actor con todas las rutas disponibles
trained_actor = train_actor_supervised(actor_model, folder_path, epochs=10)

### Entrenamiento por refuerzo (Entorno simulado)

In [ ]:
rewards = train_ddpg(agent, env, episodes=10)

# Validación modelo

## Imports

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from tensorflow.keras.models import load_model

from environment.sailing_env import SailingEnv
from environment.wind_field import MultiDayWindField
from environment.polar_diagram import PolarDiagram

import os
from environment.sailing_env import SailingEnv
from environment.wind_field import SingleDayWind
from agents.ddpg import DDPGAgent
from utils.utils import haversine, select_random_start_goal

## Parametros

In [7]:
# --- Configuración de paths ---
model_path = "models/ddpg_latest/actor_final_ep700.h5"
# route_master_path = "data/20250504_route_00.csv"
date_str = '20250504'

csv_path = "../../data/processed/nodes_bathy_wind20250504.csv" #selecionamos un fichero de malla
polar_path = "../../data/polar_diagram.csv"
graph_path = "../../data/graphs/graph_20250504.gpickle"

out_dir_drl = 'data/routes_drl'
os.makedirs(out_dir_drl, exist_ok=True)

out_dir_deter = 'data/expert_routes'
os.makedirs(out_dir_deter, exist_ok=True)

In [8]:
# Cargar modelo y datos
actor = load_model(model_path)
df = pd.read_csv(csv_path)
from environment.polar_diagram import PolarDiagram
polar = PolarDiagram(polar_path)
wind = SingleDayWind(df)


I0000 00:00:1751542642.787328 2021923 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46745 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:5b:00.0, compute capability: 8.6


In [9]:
from utils.utils import haversine

rutas = []
for i in range(7):
    start, goal = select_random_start_goal(df)
    rutas.append((start, goal))
    
    dist_km = haversine(start[1], start[0], goal[1], goal[0])  # en km
    print(f"[RUTA {i+1}] Start: {start} | Goal: {goal} | Distancia: {dist_km:.2f} km")


[RUTA 1] Start: [np.float64(40.54216383117133), np.float64(4.101076)] | Goal: [np.float64(39.77205622915002), np.float64(4.017076)] | Distancia: 85.93 km
[RUTA 2] Start: [np.float64(38.54415952873096), np.float64(3.3030760000000003)] | Goal: [np.float64(40.15821034039509), np.float64(2.7570760000000005)] | Distancia: 185.51 km
[RUTA 3] Start: [np.float64(40.10318009019184), np.float64(5.067076)] | Goal: [np.float64(39.49488722825432), np.float64(3.4710760000000005)] | Distancia: 152.20 km
[RUTA 4] Start: [np.float64(39.10497220437557), np.float64(5.067076)] | Goal: [np.float64(40.21319567926543), np.float64(4.479076)] | Distancia: 133.11 km
[RUTA 5] Start: [np.float64(39.49488722825432), np.float64(1.581076)] | Goal: [np.float64(40.377882093815096), np.float64(3.2190760000000003)] | Distancia: 170.71 km
[RUTA 6] Start: [np.float64(39.550410561341046), np.float64(1.203076)] | Goal: [np.float64(38.82512252795502), np.float64(0.825076)] | Distancia: 86.98 km
[RUTA 7] Start: [np.float64(38

## Ruta de modelo DRL

In [10]:
# Función para guardar una ruta como CSV
def save_route_to_csv(data, route_id):
    df_out = pd.DataFrame(data)
    df_out.to_csv(f"{out_dir_drl}/route_drl_{route_id:02d}.csv", index=False)

In [ ]:
from environment.reward import estimate_heel

for idx, (start, goal) in enumerate(rutas):
    print(f"[RUTA {idx}] Start: {start}, Goal: {goal}, Distancia: {haversine(start[1], start[0], goal[1], goal[0]):.2f} km")
    
    config = {
        "start": start,
        "goal": goal,
        "dt": 10,
        "max_steps": 135,
        "polar_diagram": polar,
        "wind": wind,
        "debug": False
    }

    env = SailingEnv(config)
    agent = DDPGAgent(env)
    agent.actor_model = actor

    obs = env.reset()
    done = False
    route_data = []
    step = 0

    while not done and step < config["max_steps"]:
        action = agent.actor_model.predict(obs.reshape(1, -1), verbose=0)[0]
        obs, reward, done, _ = env.step(action)

        # Calcular tiempo actual a partir del paso
        current_time = pd.to_datetime(env.wind_field.min_time) + pd.Timedelta(minutes=env.step_count * env.dt)

        # Recuperar condiciones de viento en este paso
        wind_dir, wind_speed = env.wind_field.get(env.position[0], env.position[1], current_time)
        relative_angle = (wind_dir - env.heading) % 360
        heel = estimate_heel(relative_angle, env.speed)

        route_data.append({
            "step": step,
            "lat": env.position[0],
            "lon": env.position[1],
            "heading": env.heading,
            "speed": env.speed,
            "wind_dir": wind_dir,
            "wind_speed": wind_speed,
            "heel": heel,
            "reward": reward
        })

        step += 1

    save_route_to_csv(route_data, idx)


[RUTA 0] Start: [np.float64(40.54216383117133), np.float64(4.101076)], Goal: [np.float64(39.77205622915002), np.float64(4.017076)], Distancia: 85.93 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
I0000 00:00:1751542647.032002 2022027 service.cc:152] XLA service 0x7f8e70003e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751542647.032039 2022027 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-07-03 11:37:27.045487: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751542647.071846 2022027 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1751542647.258501 2022027 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[RUTA 1] Start: [np.float64(38.54415952873096), np.float64(3.3030760000000003)], Goal: [np.float64(40.15821034039509), np.float64(2.7570760000000005)], Distancia: 185.51 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


[RUTA 2] Start: [np.float64(40.10318009019184), np.float64(5.067076)], Goal: [np.float64(39.49488722825432), np.float64(3.4710760000000005)], Distancia: 152.20 km


...

[RUTA 5] Start: [np.float64(39.550410561341046), np.float64(1.203076)], Goal: [np.float64(38.82512252795502), np.float64(0.825076)], Distancia: 86.98 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


[RUTA 6] Start: [np.float64(38.54415952873096), np.float64(-0.0569239999999999)], Goal: [np.float64(38.31858908048507), np.float64(1.707076)], Distancia: 155.68 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


: 

In [26]:
for idx, (start, goal) in enumerate(rutas):
    print(f"[RUTA {idx}] Start: {start}, Goal: {goal}, Distancia: {haversine(start[1], start[0], goal[1], goal[0]):.2f} km")
    
    config = {
        "start": start,
        "goal": goal,
        "dt": 10,
        "max_steps": 135,
        "polar_diagram": polar,
        "wind": wind,
        "debug": False
    }

    env = SailingEnv(config)
    agent = DDPGAgent(env)
    agent.actor_model = actor

    obs = env.reset()
    done = False
    route_data = []
    step = 0

    while not done and step < config["max_steps"]:
        action = agent.actor_model.predict(obs.reshape(1, -1), verbose=0)[0]
        obs, reward, done, _ = env.step(action)
        route_data.append({
            "step": step,
            "lat": env.position[0],
            "lon": env.position[1],
            "heading": env.heading,
            "speed": env.speed,
            "reward": reward
        })
        step += 1

    save_route_to_csv(route_data, idx)

[RUTA 0] Start: [np.float64(39.27234670095148), np.float64(3.6390760000000006)], Goal: [np.float64(39.21659985092966), np.float64(3.2190760000000003)], Distancia: 36.70 km


/home/agutierrez/Sailing-route-generator-for-sailboats/.venv/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
I0000 00:00:1751448743.707683 1919510 service.cc:152] XLA service 0x7fe900006180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751448743.707705 1919510 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-07-02 09:32:23.713790: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751448743.742619 1919510 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1751448743.928080 1919510 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[RUTA 1] Start: [np.float64(39.99298491065609), np.float64(3.051076000000001)], Goal: [np.float64(40.432687654175496), np.float64(1.077076)], Distancia: 174.60 km
[RUTA 2] Start: [np.float64(39.82735565397359), np.float64(3.5130760000000003)], Goal: [np.float64(40.32303156920454), np.float64(4.731076)], Distancia: 117.38 km
[RUTA 3] Start: [np.float64(38.60044108555083), np.float64(1.749076)], Goal: [np.float64(40.15821034039509), np.float64(3.5970760000000004)], Distancia: 235.00 km
[RUTA 4] Start: [np.float64(39.82735565397359), np.float64(0.195076)], Goal: [np.float64(39.937820009204), np.float64(1.287076)], Distancia: 93.98 km
[RUTA 5] Start: [np.float64(38.76901892192847), np.float64(4.731076)], Goal: [np.float64(39.49488722825432), np.float64(3.3450760000000006)], Distancia: 144.24 km
[RUTA 6] Start: [np.float64(39.4393191597462), np.float64(3.3450760000000006)], Goal: [np.float64(39.27234670095148), np.float64(1.749076)], Distancia: 138.47 km


## Imports

In [20]:
import os
import glob
import random
from datetime import datetime

import pandas as pd
import networkx as nx
from scipy.spatial import cKDTree

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

from tfm_deterministic_agent.pathfinding.search import shortest_path_with_turn_penalty
from tfm_deterministic_agent.generate_area.build_graph import PolarDiagram

## Parametros

In [21]:
beta_turn = 0.1

# 1) Carga de la lista de nodos (si la necesitas como DataFrame)
nodes_df = pd.read_csv(csv_path)

G = nx.read_gpickle(graph_path)
print(f"Nodos en G: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}")

Nodos en G: 5910, Aristas: 133514


## Ruta de modelo determinista

In [22]:
# Construir KDTree sobre los nodos del grafo
node_ids = list(G.nodes())
coords   = [(G.nodes[n]['latitude'], G.nodes[n]['longitude']) for n in node_ids]
tree     = cKDTree(coords)

In [23]:
# Instancia tu diagrama polar una vez
polar = PolarDiagram(polar_path)

for idx, (start, goal) in enumerate(rutas):
    # 1) Mapear inicio y fin a los node_id más cercanos
    _, i_start = tree.query([start[0], start[1]])
    _, i_goal  = tree.query([goal[0],  goal[1]])
    src = node_ids[i_start]
    tgt = node_ids[i_goal]

    # 2) Calcular ruta óptima con penalización de virada
    result = shortest_path_with_turn_penalty(G, src, tgt, beta_turn=beta_turn)
    path   = result['path']

    # 3) Construir la trayectoria tramo a tramo
    traj = []
    prev_hd = None
    for step, (u, v) in enumerate(zip(path[:-1], path[1:])):
        n    = G.nodes[u]
        hd_uv  = G[u][v]['heading']
        wind_dir = n['wind_dir']
        wind_spd = n['wind_speed']
        twa = abs(((hd_uv - wind_dir) + 180) % 360 - 180)
        spd_uv = polar.get_speed(twa, wind_spd)
        hd_act = prev_hd if prev_hd is not None else hd_uv

        traj.append({
            'date':           date_str,
            'route_id':       f"{date_str}_{idx:02d}",
            'step':           step,
            'latitude':       n['latitude'],
            'longitude':      n['longitude'],
            'heading':        hd_act,
            'speed':          spd_uv,
            'wind_dir':       wind_dir,
            'wind_speed':     wind_spd,
            'target_heading': hd_uv,
            'target_speed':   spd_uv
        })
        prev_hd = hd_uv

    # 4) Guardar CSV
    out_path = os.path.join(out_dir_deter, f"route_det_{idx:02d}.csv")
    pd.DataFrame(traj).to_csv(out_path, index=False)
    print(f"  → Ruta {idx:02d} guardada en {out_path}")

  → Ruta 00 guardada en data/expert_routes/route_det_00.csv
  → Ruta 01 guardada en data/expert_routes/route_det_01.csv
  → Ruta 02 guardada en data/expert_routes/route_det_02.csv
  → Ruta 03 guardada en data/expert_routes/route_det_03.csv
  → Ruta 04 guardada en data/expert_routes/route_det_04.csv
  → Ruta 05 guardada en data/expert_routes/route_det_05.csv
  → Ruta 06 guardada en data/expert_routes/route_det_06.csv


## Métricas y visualización

In [6]:
import os
from training.evaluate import comparar_rutas_detallado, mostrar_rutas_en_mapa_folium, calcular_tiempo_aproximado

In [7]:
# Carpetas donde están guardadas las rutas
carpeta_drl = "data/routes_drl"
carpeta_det = "data/expert_routes"

# Número de ruta a analizar (de 0 a 6)
ruta_id = 1  # cambia este número según la ruta que quieras


drl_path = os.path.join(carpeta_drl, f"route_drl_{ruta_id:02d}.csv")
det_path = os.path.join(carpeta_det, f"route_det_{ruta_id:02d}.csv")


In [8]:
if os.path.exists(drl_path) and os.path.exists(det_path):
    print(f"\nComparando ruta {ruta_id:02d}...\n")
    resumen = comparar_rutas_detallado(drl_path, det_path)

    print("Resultados DRL:")
    for k, v in resumen["drl"].items():
        print(f"  - {k}: {v}")

    print("\nResultados DET:")
    for k, v in resumen["det"].items():
        print(f"  - {k}: {v}")
else:
    print(f"Archivos de ruta {ruta_id:02d} no encontrados.")



Comparando ruta 01...

Resultados DRL:
  - distancia_km: 44.1314668698329
  - duracion_steps: 135
  - heel_max: 0.0219900312647223
  - heel_max_duracion: 134
  - heel_min: 0.0219900312647223
  - heel_min_duracion: 134
  - heel_prom: 0.0219900312647223
  - viradas: 0

Resultados DET:
  - distancia_km: 137.84950765071233
  - duracion_steps: 14
  - heel_max: 2.6299628838958653
  - heel_max_duracion: 1
  - heel_min: 1.4696177333397964
  - heel_min_duracion: 2
  - heel_prom: 2.059945638598972
  - viradas: 1


In [10]:
t_drl = calcular_tiempo_aproximado("data/routes_drl/route_drl_01.csv", tipo="drl")
t_det = calcular_tiempo_aproximado("data/expert_routes/route_det_01.csv", tipo="det")

print(f"🕒 Tiempo estimado DRL: {t_drl:.2f} h")
print(f"🕒 Tiempo estimado Det: {t_det:.2f} h")


🕒 Tiempo estimado DRL: 9.42 h
🕒 Tiempo estimado Det: 12.16 h


In [ ]:
mapa = mostrar_rutas_en_mapa_folium(drl_path, det_path)
mapa  # En Jupyter Notebook se renderiza automáticamente
